# Extract email rows from news sentences.
# Extract warning rows from news sentences.

# Specification

- 불필요한 문장 제거, 뉴스 문장 구분

In [1]:
import json
import pandas as pd
import re

In [32]:
with open('./data/news/201901/sentences.json') as f:
    sentences = json.load(f)

In [33]:
print(len(sentences))
print(type(sentences))

48966
<class 'list'>


In [34]:
df = pd.DataFrame.from_dict(sentences)
df.head(5)

,pid,sentence
0,b0d99f623a483d40b8c4c7fd008fbcff,(양양=뉴스1) 서근영 기자 = 1일 강원도 양양군 서면 송천리 일대 야산에서 발...
1,b0d99f623a483d40b8c4c7fd008fbcff,sky4018@news1.kr
2,b0d99f623a483d40b8c4c7fd008fbcff,"<저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>"
3,7d3ae92ebccb7228a58f7e308401010f,(양양=뉴스1) 서근영 기자 = 1일 강원도 양양군 서면 송천리 일대 야산에서 발...
4,7d3ae92ebccb7228a58f7e308401010f,sky4018@news1.kr


In [38]:
sentences[0:100]

[{'pid': 'b0d99f623a483d40b8c4c7fd008fbcff',
  'sentence': ' (양양=뉴스1) 서근영 기자 = 1일 강원도 양양군 서면 송천리 일대 야산에서 발생한 산불로 대피한 정다운마을 주민들이 대피소인 상평초등학교에서 밤을 보내고 있다. 2019.1.1/뉴스1'},
 {'pid': 'b0d99f623a483d40b8c4c7fd008fbcff', 'sentence': 'sky4018@news1.kr'},
 {'pid': 'b0d99f623a483d40b8c4c7fd008fbcff',
  'sentence': '<저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지> '},
 {'pid': '7d3ae92ebccb7228a58f7e308401010f',
  'sentence': ' (양양=뉴스1) 서근영 기자 = 1일 강원도 양양군 서면 송천리 일대 야산에서 발생한 산불로 대피한 정다운마을 주민들이 대피소인 상평초등학교에서 밤을 보내고 있다. 2019.1.1/뉴스1'},
 {'pid': '7d3ae92ebccb7228a58f7e308401010f', 'sentence': 'sky4018@news1.kr'},
 {'pid': '7d3ae92ebccb7228a58f7e308401010f',
  'sentence': '<저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지> '},
 {'pid': '539767076c4d117c6306bdf7f006e1c8',
  'sentence': ' (양양=뉴스1) 고재교 기자 = 1일 강원도 양양군 서면 송천리 일대 야산에서 발생한 산불이 한밤 중에도 강한 바람을 타고 번지고 있다. 2019.1.1/뉴스1'},
 {'pid': '539767076c4d117c6306bdf7f006e1c8', 'sentence': 'high15@news1.kr'},
 {'pid': '539767076c4d117c6306bdf7f006e1c8',
  'sentence': '<저작권자 © 뉴스1코리아, 무단전재 및 재배

In [29]:
# declare regular expression
email_check = re.compile('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')
round_bracket = re.compile('\([^)]*\)')
square_bracket = re.compile('\[[^)]*\]')
angle_bracket = re.compile('\<[^)]*\>')
single_quatation = re.compile('\'[^)]*\'')
double_quatation = re.compile('\"[^)]*\"')
str_check = re.compile('[^0-9a-zA-Zㄱ-힗]')

In [46]:
'''
TEST
'''
# text = '"test" test'
# result = double_quatation.findall(text)
# result

t = '가나다.'
t[-1]

'.'

In [61]:
'''
Returns true if the sentence contains an email.
'''
def email_checker(sentence):
    token = sentence.split(' ')
    for t in token:
        check = email_check.match(t.strip('[](),'))
        if check != None:
            return True
    return False

'''
'''
def bracket_checker(text):
    round_bracket = re.compile('\([^)]*\)')
    square_bracket = re.compile('\[[^)]*\]')
    angle_bracket = re.compile('\<[^)]*\>')
    checkers = [round_bracket, square_bracket, angle_bracket]
    for check in checkers:
        if check.match(text) != None:
            return True
    return False

def bracket_remover(text):
    round_bracket = re.compile('\([^)]*\)')
    square_bracket = re.compile('\[[^)]*\]')
    angle_bracket = re.compile('\<[^)]*\>')
    checkers = [round_bracket, square_bracket, angle_bracket]
    for check in checkers:
        if check.match(text) != None:
            print(check)
            return True
    return False

def extra_checker(text):
    if len(text) < 15:
        return 'none'
    
    extra_str = ['무단전재 및 재배포 금지', '기사문의 및 제보']
    if (text in extra_str) == True:
        return 'extra'
    return 'normal'

'''
Remove single or double quatation
Example:
'가나다' => 가나다
"가나다" => 가나다
가'나'다 => 가'나'다
"가'나'다" => 가'나'다
'''
def quatation_remover(text):
    if single_quatation.fullmatch(text) or double_quatation.fullmatch(text):
        return text[1:-1]
    return text
def make_clean_sentence(text):
    try:
        text_list = text.split(' = ')
        if len(text_list) > 1:
            clean_text = text_list[1].strip()
        else:
            clean_text = text
        # remove quotation
        clean_text = quatation_remover(clean_text)
        if clean_text[-1] != '.':
            clean_text = clean_text.split('. ')
            clean_text = clean_text[0]
        if str_check.match(clean_text[0]) != None:
            return ('extra', '')
        return ('normal', clean_text)
    except:
        print('text: ', text)
        return ('error', '')

In [30]:
# test
text = sentences[1]['sentence'].strip()
# make_clean_sentence(sentences[0]['sentence'])
sentences[1]['sentence']
email_checker(text)

True

In [76]:
t = '[한경닷컴 바로가기] [모바일한경 구독신청] 네이버에서 한국경제 채널 구독하기<자세히 보기> ⓒ 한국경제 & , 무단전재 및 재배포 금지'
round_bracket = re.compile('\([^)]*\)')
square_bracket = re.compile('\[[^)]*\]')
angle_bracket = re.compile('\<[^)]*\>')
checkers = [round_bracket, square_bracket, angle_bracket]
for check in checkers:
    if check.match(t) != None:
        print(check.match(t))

<_sre.SRE_Match object; span=(0, 24), match='[한경닷컴 바로가기] [모바일한경 구독신청]'>


In [62]:
# filtering NEW
filtered_sentences = []
for idx, element in enumerate(sentences):
    sentence = element['sentence']
    extra = extra_checker(sentence)
    if extra != 'normal':
        element['type'] = extra
        continue
    
    # email check
    email = email_checker(sentence)
    if email == True:
        element['type'] = 'email'
        continue
    
    text = sentence.strip()
    bracket = bracket_checker(text)
    if email == True:
        element['type'] = 'bracket'
        continue
        
    result = make_clean_sentence(text)
    element['type'] = result[0]
    element['cleaned_sentence'] = result[1]

In [58]:
# extract not email
sentences_normal = list(filter(lambda stc:stc['type'] == 'normal', sentences))
sentences_extra = list(filter(lambda stc:stc['type'] == 'extra', sentences))
sentences_bracket = list(filter(lambda stc:stc['type'] == 'bracket', sentences))
sentences_none = list(filter(lambda stc:stc['type'] == 'none', sentences))
sentences_email = list(filter(lambda stc:stc['type'] == 'email', sentences))

In [59]:
# 38092
print(len(sentences_normal))

# 4076
print(len(sentences_extra))

# 0
print(len(sentences_bracket))

# 2395
print(len(sentences_none))

# 4403
print(len(sentences_email))

38092
4076
0
2395
4403


In [60]:
for idx, s in enumerate(sentences_extra):
    print(idx, '>>>',  s['sentence'], '\n')

0 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

1 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

2 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

3 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

4 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

5 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

6 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

7 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

8 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

9 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

10 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

11 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

12 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

13 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

14 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

15 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

16 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

17 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

18 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

19 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

20 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

21 >>>  【양양=서정욱기자】1일 오후 4시12분께 강원도 양양군 서면 송천리 야산에서 불이나 산림·소방당국이 진화에 나섰다.  

22 >>> [경향신문] ㆍ지난달부터 건조특보 발효 중…방화선 구축해 밤샘 진화 

23 >>> [한국경제TV 이영호 기자]1일 오

1006 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

1007 >>> “의원님들이 예천 망신 다 시켜놨으니까 전원 사퇴하는 게 정답입니다.” 

1008 >>> [아시아경제 한진주 기자] 광화문역 출구 앞 도로에서 택시에 화재가 발생해 택시기사가 화상을 입고 병원으로 이송됐다. 소방당국은 택시 기사가 분신을 시도한 것으로 보고 있다. 

1009 >>> [한국경제TV 김주리 기자] 

1010 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

1011 >>> [서울경제] 9일 오후 6시 3분경 서울 광화문광장 인근 도로에서 택시에 갑작스런 화재가 발생했다. 

1012 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

1013 >>> [한국경제TV 김주리 기자] 

1014 >>> [편집자주] "연탄재 함부로 발로 차지 마라, 너는 누구에게 한번이라도 뜨거운 사람이었느냐"(안도현-너에게 묻는다 중에서). 2019년의 세상 사람들에게 연탄은 말한다. 다 식어가는 나의 온기라도 절실하고 그리운 사람들이 있다고. 영하 10도 이하로 내려가는 달동네, 쪽방촌 그 냉골을 버티게 하는 2019년 연탄의 얘기를 들어봤다. 

1015 >>> [편집자주] "연탄재 함부로 발로 차지 마라, 너는 누구에게 한번이라도 뜨거운 사람이었느냐"(안도현-너에게 묻는다 중에서). 2019년의 세상 사람들에게 연탄은 말한다. 다 식어가는 나의 온기라도 절실하고 그리운 사람들이 있다고. 영하 10도 이하로 내려가는 달동네, 쪽방촌 그 냉골을 버티게 하는 2019년 연탄의 얘기를 들어봤다. 

1016 >>> ◇"코치-선수'주종 관계'…성폭력 사각지대" 

1017 >>> ◇"전수 조사·외부전문가 참여 예방 기관 만들어야…조기 교육 필수" 

1018 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

1019 >>> [ 박진우 기자 ] 유튜버 양예원 씨의 사진을 유출하고 추행한 혐의로 기소된 40대 남성이 1심에서 실

2006 >>> '남산 3억원 의혹'은 2008년 2월 남산자유센터 주차장에서 신한 측이 이 전 대통령 친형인 이상득 전 한나라당 의원 측에게 3억원을 전달했다는 의혹이다. 이는 2010년 9월 라응찬 전 신한금융 회장이 신 전 사장과 경영권 분쟁을 벌이다 세간에 알려졌다. 

2007 >>> [디지털뉴스국 노경민 인턴기자] 

2008 >>> [한경닷컴 바로가기] [모바일한경 구독신청] 네이버에서 한국경제 채널 구독하기<자세히 보기> ⓒ 한국경제 & , 무단전재 및 재배포 금지 

2009 >>> [경향신문] 카지노에서 노름을 하다 돈 문제로 다투던 고향친구를 살해한 30대 중국인에 대해 경찰이 구속영장을 신청했다. 

2010 >>> [경향신문] 지난 11일 경남 통영 앞바다에서 화물선과 부딪친 뒤 전복된 낚싯배 ‘무적호’가 공해에서 불법으로 영업한 것으로 드러났다. 

2011 >>> [아시아경제 이설 기자] 음주운전으로 적발된 현직판사가 법원의 벌금형에 불복해 정식 재판을 청구했다. 

2012 >>> [디지털뉴스국 노경민 인턴기자] 

2013 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

2014 >>> (서울=연합뉴스) 전 유도선수 신유용 씨가 최근 자신의 소셜네트워크서비스(SNS)를 통해 고교 재학 시설 유도부 코치로부터 수년간 성폭행을 당했다고 주장했다. 신 씨는 지난해 3월 성폭행을 돈으로 회유하려는 코치의 모습을 보고 서울 방배경찰서에 고소했다. 8개월 뒤인 11월엔 "현재 사건은 수사 촉탁으로 인해 시한부 기소중지가 이뤄졌으며 서울 중앙지검에서 피의자 관련 수사가 재개될 것으로 보인다"라며 "이번 사건이 어떻게 진행되는지 추후 말씀드리겠다"고 밝혔다. 2019.1.14 [신유용 페이스북 캡처] 

2015 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

2016 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>  

2017 >>> <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>

3117 >>> [디지털뉴스국 노경민 인턴기자] 

3118 >>> [아이뉴스24 권준영 기자] 미국 그랜드캐년을 여행하던 25살 청년이 20일째 사경을 헤매고 있다. 

3119 >>> [디지털뉴스국 노경민 인턴기자] 

3120 >>> 【청도=김장욱 기자】함께 술을 마시고 말다툼을 하다 후배를 흉기로 찔러 숨지게 한 50대 남성이 경찰에 붙잡혔다.  

3121 >>> [의정부=이데일리 정재훈 기자] 의료계에서 신의 직장으로 알려진 국립암센터의 총체적 채용비리가 경찰 수사결과, 사실로 드러났다. 

3122 >>> [아시아경제 김흥순 기자] 정부가 지도자의 폭력이나 성폭력 등으로 고통 받는 선수들을 보호하기 위해 특별사법경찰관(특사경) 배치를 추진한다. 체육계 비위에 대한 수사부터 징계까지 빠르고 엄정하게 처리하기 위한 조치다. 

3123 >>> ◇의대생 절반이 언어폭력 경험…여학생 18.3% 신체적 성희롱 겪어실태조사 결과 의과대학 학생들은 언어폭력부터 신체적 폭력, 음주 강요까지 다양한 폭력과 강요를 경험한 것으로 조사됐다. 

3124 >>> ◇피해사실 신고 학생 단 3.7%…“신고해도 아무 소용 없을 것”이처럼 인권 침해가 만연하지만 대학 및 병원에 피해 사실을 신고한 학생은 단지 3.7%에 불과한 것으로 드러났다. 더욱이 신고 학생들조차 학교 차원에서 가해자 처벌 등 제대로 된 조치를 취하지 않고 피해자를 비난하는 등 2차 피해를 경험하면서 신고 결과에 대부분 만족하지 못했다고 답했다. 

3125 >>> [경향신문] 같은 곳에서 같은 일을 하는데도 차별 대우를 받는다. 하도급 노동자는 차량을 못가져오는데 정규직 노동자는 자유롭게 출입한다. 목욕탕 개인사물함의 종류조차 다르다.  

3126 >>>  [이데일리 김은총 기자] 제주에서 일주일째 행방이 묘연했던 30대 남성이 공개수사 하루만에 무사히 발견돼 귀가하게 됐다. 

3127 >>> [아이뉴스24 권준영 기자] 극단적 선택을 암시하는 글을 남기고 제주도로 내려온 후 행방이 묘연했던 30대 

In [ ]:
# 502c80f96a0b44dd84a9581c6bcffcf0 / a5216e0834740a1b6be20a392a528e30 / aa583a688beeb7ec6ce4d484f1f1c5ca
# e9341587b10c0cb28c4aa4ccaad10944 / 1df30f2c433fa9491e7fd8d8972819d4 / f5fb4e59c162a8b4adfabe3c9745b05f
# 2dd92e7dba8b2e44cf35b15bf70901f0 / 0d2d495f0aa3903ed09f765dfb60c85d / a0fd930cea2666de2623b0f9586100dd
# 3c876c23863da3151536b9a21e98464f / f24a4af5791fa55db3acedab450bf1d8 / 461297b28c65ac19a4ace48dad5c3ab8
# 668e65bea1e610c8445a46a24c528fbf / 7ce8e32ec8a9a571dd3d25940c91458e / b9d04be32d8ea5a272a0a128ea311dbe
# 7da8a3eb29441f579a0d7083f3628d8d / c64bed8028bda69453871b8ea681bb8e / 50c6544481ee834a60391b16b5b18b57
# 02932f1436cd8f8628c03b7e2d1d4bdb / 15c3340a66356cde5b167460dc98ebde / a376d3bcd6d2aa721a742be05668e231
# 7565abe83a08550687c1bb428aa63801 / fbb89185ced618cb4cc33ec51efd87a2 / 6d437ff269df5f787aef91d70e8c0f38
# 1ee93495ff36718e0121ae8bc583d02e / c7b2b91b02a2eb7adf7dec7fd2721b9a / 7d3780d2ace9e566a59ba4141a181e08
# cf877710bef93d858e259514ae62bbca / 28dd284ad26d1da601a6b5e78c9fc7ed / e6dac9e501efa2b7bcccb9f1042f8a7c
# fff39f9f10f127f29ab4fcc5812cd948 / 204a430938e607eaaecaf1620477b4c1 / a9338e16c91ae57fb1ed7966f69a1b84
# 573391debc8b6d04032c68fd4e96077d / 1dbb4973e77d5751e844b87db0eb624f / 82ade7d1b07256c6fe6b02b5788ecf48
# ebdc474722475259885f8d6ded61839e / 3cf1d206d57d6adf14bca034b92215da / 73df6c30f8eb54d6396459d6a3666dfc
# e2c82f754cad10f7088d7439f460acc2 / 5a07304724e5a8312e4f143a56cf0ee1 / 5ba380994a642edf372e53c9ca5468aa
# 281c9f274525eb0d0cf2c1c5aa85fad8 / a3e6b15454a5b1fef11b92d0111ce238 / c389a3248f23046f50919b541fa60d91
# 78a36e5d7394881e523d736952fda659 / e164bb1cdd3dbb142b0a0c51f198e7eb / aa2c0511c8d2ba33401507c1ea3493e9
# 73e577c0b3eae68c1beb939850af2a26 / 205423a4c36a75903d03383895a4cc83 / 966e5f2320d5e28426950ece0df425c7
# 4c25fc2e23c5aa2360c182ad4e2119ff / f888aa95bba4b11c65c03f0c8fd7a12e / c97bb721b7207e11d3404312ffa947e5
# 30a67a99fd2efb36d67b743dca009f5f / 30a67a99fd2efb36d67b743dca009f5f / 0cd7d57eca63a457a2b30787bfbaf35f
# e676423569b8562ee825abbaa88feef7 / b9ef07e5fdf02cb42314fb8504a1ac9a / f5536d253ebb74c21a2b98e93ab96854
# 8b39085a91e7812681d4e8f9ebcb527a / 7f4071c07bccc479dce317d9eef0660a / acc258dd84bdcc4ddec1bc0a82d7d1a8
# 2b514804aed6adda1b5fdf1e1f9253c4 / 2e0139984685915fd8712effce1fc0fd / 6a0d4453a6def2b16b422b665a108118
# 74d96b34441b5296bc01eeae5844eb06 / 465133171ce9113f404f5752b070e2dc / a382967040e035ef0ce2f15cba0ef664
# 070224cfa1b84e088141d934aced6cf1 / 6f9ba40475cd9337a2e576b62ff79bc4 / 008039ffb605d5c4c8b8466f75e05839
# 66195724300b5387018132fcc2ad640b / d267e43dc9594c2ef2c94d00d850a068 / 77e46ffc81f5a50288260dd24f04454a
# 68a14b8b939d58b2c2da8a66085db915 / efc80367b391fa8d1af34270b1a234fa / 6ba599312f3ede253aaf569ed22fa38c
# d65b5344ff628ea4c97f04f21a46d55d / 76b985a7cae6d811f5e1858179c740c8 / 70a7e8a010c026fd060fe74fcd32c482
# 2b381e68906e0f514dda9441485828ba / 96830139c4167bb5f3dd8abda7be07c7 / df5d01371337da4cfbb3091c08d8cc71
# 57d375c2e8d1c29e7a7e01bf911d697d / 714ff8207c265ab39ef83773bbc9154c / 5988841299504ddda9d752f995b182a2
# d637cb23b161373ce2a89c5c2f00f24a / b66c8b75e8dbf07075d802c1e77831c2 / 5c25093aa2c96eb2cb6da7329d2771b6
# 70b81fa2fb955210820e6e3e5f02c51a / 04e1d6ac9c65b51b050c08e9d3564dd1 / 21ef453143cdcbafb084d125fa491105
# 360c5e6aff755f4166969b89e730bcf1 / 69ac9b3075868c3c6f07dd925b5e7f31 / 6e6ae35799c6b8d9204aa1206790df1f
# 9861ff62f9fcca0e1adb81d0d5b67276 / 6885d5ecb594d6126b2e0964440fb4c0 / dfaf602bf5cde7eb7b9e8928ce05e86d
# 675ac6465e8654437829309835394a3b / 22036e715e70f033073855cb126d612d / 4b7c8eec0405841ddb893f81081ee8b1
# 859e777e653e95cc0b8b0d0dc83ae11d / b4b5856b28257b821a05499b19649eb7 / 4df10a78494b38bdcbff160ccc14b82d
# 516cb8c6c2120c521e52e9d83af495a3 / eaabf00ffbc84c974e82465032431bdb / a0e595bde0f6d207a34fff1d19f1c9f5
# 304cc780cfeacbfdbef31f6d30ff7278 / 5805a2ce45e13a6a28365a378661d34b / e8282bfaf0c8d8d0849ccdb4fd7b783d
# 4475cce384c23d0d3005d8417c39e86f / 3b7786c298493d2fbd061584ca008ed5 / a200443b8e29139882fc64eeb712247b
# c082486d2cf5a5166cccc5a5f83e59f7 / 54b75e09ebef9c03e237e1462da7578a / 163e71977ad3eadd21caebd172b2d88f
# ac4f96a1ee56cb376e33310342c7a185 / 7526f02d69a73a6f5bbb0f7695bcb776 / fa9c308cd84fe0dfc474a74907348237
# 10c42320f35a154bcae7982dfad0665a / 345863b2659a6bfbdd6fc4a8c4e00989 / b8b2d23262f3f339e0a8b5997bbffde8
# 59fb1040b647d37195255254dabb0920 / 360be099f5d7b742ab67abac1b1af455 / 973e7ab5f4ef3f9fbdc0687c60d1d5d5
# 0be6cdf7e82d646ccddb591550a7eecb / fc6e5b12c7406e6f62492ce97716052a / b00614038d462ebf0c80e6111261e139
# cfe0b936583ecced32fcbae0697208a4 / 81eabbd71b46c0ac707c083d5aa9278b / b371b7bff4f30790139ef195ab7263fb
# 33d5b92f36ec758c582ba1b1a4bb6989 / f48fd618cf6864d912dc22d54b1593ae / 7207c5be36854cd0ed9b21816e6b36b9
# f4698bdf5c876499f2460181337fa249 / 8ea5bee858a763ccb202181a3741ebef / 184d7db700a24d5e0949ded0ec04d89f
# e580f98ae9a9ec568f203c80ec49a918 / 80f3e1c873cb0efa0737082b1d4facea / 095b9f567da16e144e5030a07e7caac4
# 3811ad052fe6a4ec225be1bd28ec5b18 / b33c6ac6a4e95a1ff13e94b8f76da153 / 5f9618dc86ec66a34eaa86c7dc668031
# b973feae35c97dc35d4c7a8a0ab05cb8 / 
n = 0
for s in sentences_email:
    n += 1
    print(n, ': ',s['pid'], '>>>',  s['sentence'])

In [77]:
sentences_normal = list(filter(lambda stc:stc['type'] == 'normal', sentences))

In [79]:
for s in sentences_normal:
    del s['sentence']

In [80]:
sentences_normal[0]

{'pid': 'b0d99f623a483d40b8c4c7fd008fbcff',
 'type': 'normal',
 'cleaned_sentence': '1일 강원도 양양군 서면 송천리 일대 야산에서 발생한 산불로 대피한 정다운마을 주민들이 대피소인 상평초등학교에서 밤을 보내고 있다'}

In [81]:
with open('./data/news/201901/sentences_normal.json', 'w', encoding='utf-8') as make_file:
    json.dump(sentences_normal, make_file, indent="\n")